In [1]:
import pandas as pd
import pyensembl
import numpy as np

In [2]:
#data.to_csv("~/projects/BIO/novel_junction.20230512.tsv",sep="\t",index=False)
data=pd.read_csv("~/projects/BIO/proteomics_MU_junction_all.20230529.tsv",sep="\t")

In [3]:
all_intron=pd.read_csv("/home/ls/rachelcw/projects/BIO/annontation_code.20221225_all_introns.bed.gz",sep="\t",header=None,compression="gzip",usecols=[0,1,2,3,4,5,6,8],names=["chr","start","end","gene","gene_id","strand","transcript_id","annotation"])
protein_intron=all_intron[all_intron["annotation"]=="protein_coding"]

/home/ls/rachelcw/miniconda3/envs/bio/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# in order to get the genes by individual.
def get_gene_list(genelist):
    data_gene_list=[]
    for value in genelist:
        value = value.split(',')
        data_gene_list.extend(value)
        
    return list(set(data_gene_list))

*******************************************************
prepare data table- merge leafcutter ds results 

In [48]:
# mutated VS unmutated SF3B1 - group 2 - M+U CLL

# read in the data
cluster = pd.read_csv('~/projects/LEAFCUTTER/DS/DS.five_percent/analysis.20230512/fdr0.05/filtered.a2.20230512_cluster_significance.txt',sep=" ")
effect_size=pd.read_csv('~/projects/LEAFCUTTER/DS/DS.five_percent/analysis.20230512/fdr0.05/filtered.a2.20230512_effect_sizes.txt',sep=" ")

In [63]:
# filtered out junction mut(psi) > unmut(psi) --> delta psi < 0
mutated=effect_size[effect_size["deltapsi"]<0]

In [ ]:
mutated[["chr","start","end","cluster"]]=mutated["intron"].str.split(":",expand=True)
mutated["start"]=mutated["start"].astype(int)
mutated["end"]=mutated["end"].astype(int)
cluster[["chr","cluster"]]=cluster["cluster"].str.split(":",expand=True)
mutated=mutated.merge(cluster,on=["chr","cluster"],how="left")


In [65]:
data=mutated.filter(items=["intron","chr","start","end","genes","cluster","p.adjust"])
data.dropna(axis=0,inplace=True)

In [66]:
# gtf=pd.read_csv("/private/resources/gencode19_noChrPrefix_mitoMT.gtf",sep="\t", comment='#', header=None)
# all_intron=pd.read_csv("/home/ls/rachelcw/projects/BIO/annontation_code.20221225_all_introns.bed.gz",sep="\t",header=None,compression="gzip")

In [5]:
# filter out intron that are not in the data
data_gene_list=get_gene_list(data['genes'].unique())
intron_in_data=protein_intron[protein_intron["gene"].isin(data_gene_list)]

In [6]:
genome = pyensembl.Genome(
    reference_name='GRCh37',
    annotation_name='my_genome_lab',
    gtf_path_or_url='/home/ls/rachelcw/projects/protein_coding.gtf')
genome.index()

In [71]:
data["novel"]=pd.NA
data["transcript_id"]=pd.NA
for gene in data_gene_list:
    # data_gene=data[data["genes"].str.contains(gene)]
    # intron_gene=intron_in_data[intron_in_data["gene"].str.contains(gene)]
    for index,row in data.iterrows():
        if ((intron_in_data["start"]==row["start"])&(intron_in_data["end"]==row["end"])).any():
            data.loc[index,"novel"]="known"
            trans_id=intron_in_data.loc[(intron_in_data["start"]==row["start"])&(intron_in_data["end"]==row["end"]),"transcript_id"]
            data.loc[index,"transcript_id"]=','.join(trans_id)
        elif (intron_in_data["start"]==row["start"]).any():
            data.loc[index,"novel"]="start"
            trans_id=intron_in_data.loc[intron_in_data["start"]==row["start"],"transcript_id"]
            data.loc[index,"transcript_id"]=','.join(trans_id)
        elif (intron_in_data["end"]==row["end"]).any():
            data.loc[index,"novel"]="end"
            trans_id=intron_in_data.loc[intron_in_data["end"]==row["end"],"transcript_id"]
            data.loc[index,"transcript_id"]=','.join(trans_id)
        else:
            data.loc[index,"novel"]="novel"
            transcript_of_intron=genome.transcript_ids_at_locus(contig=row.chr.replace("chr",""),position=row.end)
            if len(transcript_of_intron)>0:
                data.loc[index,"transcript_id"]=','.join(transcript_of_intron)

        

In [76]:
data[['cluster','strand']]=data['cluster'].str.rsplit('_', expand=True, n=1)

In [75]:
data.dropna(axis=0,inplace=True)

In [8]:
# Duplicate rows based on comma-separated values
new_rows = []
for index, row in data.iterrows():
    values = row['genes'].split(',')
    if len(values) > 1:
        for i, value in enumerate(values):
            new_row = row.copy()
            new_row['genes'] = value
            new_rows.append(new_row)
    else:
        new_rows.append(row)

# Create updated DataFrame
data = pd.DataFrame(new_rows)


In [7]:
gene_dict={key: value for key, value in zip(protein_intron["gene"].unique(),protein_intron["gene_id"].unique())}

In [9]:
data["gene_id"]=data["genes"].map(gene_dict)

In [10]:
# drop gene that arn't "protein coding" 
data.dropna(axis=0,inplace=True)

In [16]:
data.to_csv("~/projects/BIO/proteomics_MU_junction_all.20230530.tsv",sep="\t",index=False)

**********************************************************

In [13]:
# filter out intron that are known
novel_intron=data[data["novel"]!="known"]

In [14]:
gene_list=get_gene_list(novel_intron["genes"].unique())

create dictionary: key-transcript id, values- exons location of the transcript

In [15]:
transcript_exons=dict()
for index, row in novel_intron.iterrows():
    transcript=row["transcript_id"].split(",")
    for t in transcript:
        if t not in transcript_exons:
            exons=genome.exon_ids_of_transcript_id(t)
            locus=[genome.locus_of_exon_id(exon).to_tuple() for exon in exons]
            transcript_exons[t]=locus            
            

In [16]:
from types import MappingProxyType
# create the dictionary constant
transcript_exons=MappingProxyType(transcript_exons)

In [52]:
transcript_exons_updated=transcript_exons.copy()

* "novel"= start - we know the end of the exon 
so we want to find the next exon that is the closest to the end of the intron(start of the next exon) and replace the exon start with the intron end

* "novel"= end - we know the start of the exon 
so we want to find the previous exon that is the closest to the start of the intron(end of the previous exon) and replace the exon end with the intron start

* "novel" = novel intron -
the intron is novel so we want to find the closest exons that the intron is between.

In [17]:
def get_novel_gene_locus(transcript, start, end, novel):
    locus=transcript_exons[transcript]
    if novel=="start":
        start_exon=np.array([l[1] for l in locus])
        index_exon=np.argmax(start_exon>end)
        if start==locus[index_exon][2]:
            index_exon=index_exon+1
        new_exon_locus=tuple([locus[index_exon][0],end,locus[index_exon][2],locus[index_exon][3]])
        locus[index_exon]=new_exon_locus
    elif novel=="end":
        end_exon=np.array([l[2] for l in locus])
        index_exon=np.argmin(end_exon<start)
        if end==locus[index_exon][1]:
            index_exon=index_exon-1
        new_exon_locus=tuple([locus[index_exon][0],locus[index_exon][1],start,locus[index_exon][3]])
        locus[index_exon]=new_exon_locus
    elif novel=="novel":
        start_exon=np.array([l[1] for l in locus])
        end_exon=np.array([l[2] for l in locus])
        index_start=np.argmax(start_exon>end)
        if start==locus[index_start][2]:
            index_start=index_start+1
        index_end=np.argmin(end_exon<start)
        if end==locus[index_end][1]:
            index_end=index_end-1
        new_start=tuple([locus[index_start][0],end,locus[index_start][2],locus[index_start][3]])
        new_end=tuple([locus[index_end][0],locus[index_end][1],start,locus[index_end][3]])
        locus[index_start]=new_start
        locus[index_end]=new_end
    return locus

In [18]:
transcript_gene_id=dict()
for index, row in novel_intron.iterrows():
    transcript=row["transcript_id"].split(",")
    for t in transcript:
        transcript_gene_id[t]=row["gene_id"]

In [37]:
# file of the original exons #
file_path="/home/ls/rachelcw/projects/BIO/cll_sf3b1_proteomics_original_draft.txt"
with open(file_path,'w') as file:
    for t_id,locus in transcript_exons.items():
        for loci in locus:
            file.write(f'{loci[0]}\t{loci[1]}\t{loci[2]}\t{loci[3]}\t{t_id}\t{transcript_gene_id[t_id]}\n')

In [62]:
# file_path="/home/ls/rachelcw/projects/BIO/cll_sf3b1_proteomics_novel.txt"
# with open(file_path,'w') as file:
#     for intron,t_id in intron_transcript.items():
#         for t in t_id:
#             for t_id,locus in t.items():
#                 for loci in locus:
#                     file.write(f'{loci[0]}\t{loci[1]}\t{loci[2]}\t{loci[3]}\t{t_id}\t{transcript_gene_id[t_id]}\t{intron}\n')

In [21]:
file_path="/home/ls/rachelcw/projects/BIO/cll_sf3b1_proteomics_novel.txt"
with open(file_path,'w') as file:
    for index, row in novel_intron.iterrows():
        transcript=row["transcript_id"].split(",")
        for t in transcript:
            locus=get_novel_gene_locus(t,row["start"],row["end"],row["novel"])
            for loci in locus:
                file.write(f'{loci[0]}\t{loci[1]}\t{loci[2]}\t{loci[3]}\t{t}\t{transcript_gene_id[t]}\t{row["intron"]}\n')

In [ ]:
# # option to frame shift- need to change the modulo to the len of the gene until the new locus # #
# start_novel_intron=novel_intron[novel_intron["novel"]=="start"]
# # start_novel_intron["frameshift"]=0
# for index, row in start_novel_intron.iterrows():
#     transcript=row["transcript_id"].split(",")
#     t_frame=[]
#     for t in transcript:
#         start_exon=np.array([l[1] for l in transcript_exons[t]])
#         print(row["end"])
#         print(start_exon)
#         index_exon=np.abs(start_exon-row["end"]).argmin()
#         print(index_exon)
#         print("old",transcript_exons[t][index_exon])
#         new_exon_locus=tuple([transcript_exons[t][index_exon][0],row["end"],transcript_exons[t][index_exon][2],transcript_exons[t][index_exon][3]])
#         transcript_exons_updated[t][index_exon]=new_exon_locus
#         if (transcript_exons[t][index_exon][2]-transcript_exons[t][index_exon][1]-1)%3!=0:
#             print("new",new_exon_locus)
#             t_frame.append(t)
#     start_novel_intron.loc[index,"frameshift"]=','.join(t_frame)
        
        


In [57]:
# # my tryint to create a dictionary of {intron(junction):{t_is:[exons locus]}} ##

# intron_transcript=dict()
# for index, row in novel_intron.iterrows():
#     transcript=row["transcript_id"].split(",")
#     intron_transcript[row["intron"]] = [{t: transcript_exons[t]} for t in transcript]

# start_novel_intron=novel_intron[novel_intron["novel"]=="start"]
# # start_novel_intron["frameshift"]=0
# for index, row in start_novel_intron.iterrows():
#     transcript=row["transcript_id"].split(",")
#     for i,t in enumerate(transcript):
#         start_exon=np.array([l[1] for l in transcript_exons[t]])
#         index_exon=np.argmax(start_exon>row["end"])
#         if row["start"]==transcript_exons[t][index_exon][2]:
#             index_exon=index_exon+1
#         new_exon_locus=tuple([transcript_exons[t][index_exon][0],row["end"],transcript_exons[t][index_exon][2],transcript_exons[t][index_exon][3]])
#         # print(intron_transcript[row["intron"]][i][t][index_exon])
#         intron_transcript[row["intron"]][i][t][index_exon]=new_exon_locus
#         # print(intron_transcript[row["intron"]][i][t][index_exon])
#         # print(t)
#         # print(row["end"])
#         # print(start_exon)
#         # print(index_exon)
#         # print("old",transcript_exons[t][index_exon])
#         # print(transcript_exons[t])
#         # print("new",new_exon_locus)
           
# end_novel_intron=novel_intron[novel_intron["novel"]=="end"]
# # start_novel_intron["frameshift"]=0
# for index, row in end_novel_intron.iterrows():
#     transcript=row["transcript_id"].split(",")
#     # t_frame=[]
#     for i,t in enumerate(transcript):
#         end_exon=np.array([l[2] for l in transcript_exons[t]])
#         index_exon=np.argmin(end_exon<row["start"])
#         if row["end"]==transcript_exons[t][index_exon][1]:
#             index_exon=index_exon-1
#         new_exon_locus=tuple([transcript_exons[t][index_exon][0],transcript_exons[t][index_exon][1],row["start"],transcript_exons[t][index_exon][3]])
#         intron_transcript[row["intron"]][i][t][index_exon]=new_exon_locus
#         # print(end_exon<row["start"])
#         # print(t)
#         # print(row["start"])
#         # print(end_exon)
#         # print(index_exon)
#         # print("old",transcript_exons[t][index_exon])
#         # print(transcript_exons[t])
#         # print("new",new_exon_locus)

# novel_novel_intron=novel_intron[novel_intron["novel"]=="novel"]
# # start_novel_intron["frameshift"]=0
# for index, row in novel_novel_intron.iterrows():
#     transcript=row["transcript_id"].split(",")
#     # t_frame=[]
#     for i,t in enumerate(transcript):
#         start_exon=np.array([l[1] for l in transcript_exons[t]])
#         end_exon=np.array([l[2] for l in transcript_exons[t]])
#         index_start=np.argmax(start_exon>row["end"])
#         if row["start"]==transcript_exons[t][index_start][2]:
#             index_start=index_start+1
#         index_end=np.argmin(end_exon<row["start"])
#         if row["end"]==transcript_exons[t][index_end][1]:
#             index_end=index_end-1
#         new_start=tuple([transcript_exons[t][index_start][0],row["end"],transcript_exons[t][index_start][2],transcript_exons[t][index_start][3]])
#         new_end=tuple([transcript_exons[t][index_end][0],transcript_exons[t][index_end][1],row["start"],transcript_exons[t][index_end][3]])
#         intron_transcript[row["intron"]][i][t][index_start]=new_start
#         intron_transcript[row["intron"]][i][t][index_end]=new_end

In [ ]:
# pyensembl.search.find_nearest_locus(start, end, loci)
# Finds nearest locus (object with method distance_to_interval) to the interval defined by the given start and end positions. Returns the distance to that locus, along with the locus object itself.

In [49]:
non_protein=get_gene_list(novel_intron.loc[novel_intron["transcript_id"]=='',"genes"].unique())

In [60]:
len(novel_intron.iloc[191]["transcript_id"])

0

In [54]:
protein_intron.loc[protein_intron["gene"].isin(non_protein),"gene"].unique()

array(['PLGLB2'], dtype=object)

In [59]:
genome.transcript_ids_at_locus(contig="2",position=88037624)

[]

In [21]:
genome.transcript_ids_of_exon_id('ENSE00000728678.1')

ValueError: No results found for query:

            SELECT distinct transcript_id
            FROM transcript
            WHERE exon_id = ?
        
with parameters: ['ENSE00000728678.1']

In [62]:
genome.transcript_ids_at_locus(contig="10",position=131960844,end=131964771,strand="+")

['ENST00000331244.5', 'ENST00000368644.1', 'ENST00000481034.1']

In [10]:
genome.exon_ids_of_transcript_id("ENST00000368644.1")

['ENSE00002174390.1',
 'ENSE00000728668.1',
 'ENSE00003477570.1',
 'ENSE00000728676.1',
 'ENSE00000728678.1',
 'ENSE00000728681.1',
 'ENSE00000728683.1',
 'ENSE00000728687.1',
 'ENSE00000728691.1',
 'ENSE00000577957.1',
 'ENSE00001217321.1',
 'ENSE00001217252.2']